In [ ]:
!pip install ensemble_boxes

In [ ]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO
import os

In [ ]:
# =====================================================
# TODO : Ensemble할 실험 폴더명 입력
input_exp = [
    '[aug]offline_mosaic_v1',
    '[aug]randomflip_v1',
    '[aug]battery_mosaic_cutmix_v1',
]
# TODO : test.json 경로 설정
annotation = '../../../../dataset/test.json'
# =====================================================

In [ ]:
submission_files = []
for exp in input_exp:
    result_dir = os.path.join(exp, 'results')
    for file_name in os.listdir(result_dir):
        if file_name.startswith('.'):
            continue
        submission_files.append(os.path.join(result_dir, file_name))

In [ ]:
submission_files

In [ ]:
submission_df = [pd.read_csv(file) for file in submission_files]
image_ids = submission_df[0]['image_id'].tolist()

coco = COCO(annotation)

prediction_strings = []
file_names = []

iou_thr = 0.5            # TODO : iou threshold 설정
skip_box_thr = 0.0001    # TODO : skip iou threshold 설정

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        # boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [ ]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

save_folder = '[results]ensemble'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    
submission.to_csv(save_folder+'/submission_ensemble.csv')
submission.head()